<a href="https://colab.research.google.com/github/traderjoevitamins/librarian/blob/main/librarian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [ ]:
!pip install --upgrade 'tensorflow_data_validation[visualization]<2'
!pip install -U scikit-learn
!pip install tensorflow-text
!pip install bert-tensorflow
!pip install bert-for-tf2
!pip install transformers
!pip install torch


In [24]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_data_validation as tfdv
import pandas as pd
import nltk
import tensorflow_hub as hub
import nltk
import sklearn
import bert
import math
import torch

from transformers import BertTokenizer, BertModel
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from bert import tokenization
from tokenizers import Tokenizer
from tokenizers.models import BPE
from transformers import AutoTokenizer

#from bert.tokenization import FullTokenizer
#FullTokenizer = bert.bert_tokenization.FullTokenizer


nltk.download('punkt')
nltk.download('punkt')
print('TF version:', tf.__version__)
print('NLKT version:', nltk.__version__)
print('Hub version:', hub.__version__)
print('TFDV version:', tfdv.version.__version__)


TF version: 2.13.0
NLKT version: 3.8.1
Hub version: 0.14.0
TFDV version: 1.14.0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [74]:
goodreads = pd.read_csv('/content/finalgoodreads.csv')
goodreads['enjoyed'] = 1
goodreads = goodreads[['Title', 'Author', 'ISBN13', 'synopsis', 'enjoyed']]
goodreads_synopsis = goodreads['synopsis']

nyt1 = pd.read_csv('/content/nyt1.csv')
nyt2 = pd.read_csv('/content/nyt2.csv')
nyt3 = pd.read_csv('/content/nyt3.csv')
nyt = pd.concat([nyt1, nyt2, nyt3], axis = 0)
nyt_synopsis = nyt['synopsis']




In [80]:
nyt.to_csv('nyt.csv')



In [ ]:
goodreads_synopsis = goodreads_synopsis.str.replace('<p class="null1">','').replace('</p>\n<p>', '')
goodreads_synopsis = goodreads_synopsis.str.replace('<p>', '').replace('<', '').replace('>', '')
goodreads_synopsis = goodreads_synopsis.str.replace('<BR>', '').replace('</p><h3>', '').replace('>', '')
goodreads_synopsis = str(goodreads_synopsis)


nyt_synopsis = nyt_synopsis.str.replace('<p class="null1">','').replace('</p>\n<p>', '')
nyt_synopsis = nyt_synopsis.str.replace('<p>', '').replace('<', '').replace('>', '')
nyt_synopsis = nyt_synopsis.str.replace('<BR>', '').replace('</p><h3>', '').replace('>', '')
nyt_synopsis = nyt_synopsis
nyt_synopsis = str(nyt_synopsis)

670

382
363
327
1072


In [20]:
#tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True) # Making the model return all hidden-states.
model.eval()


In [ ]:
## tms to do
# - [ ] fix whatever is going on with nyt df
# - [ ] send add ur embeddings to corrosponding df
# - [ ] predict!

In [54]:
gtokenized_text = []
gsegments_ids = []
gtokens_tensor = []
gsegments_tensors = []

for i, syno in enumerate(goodreads_synopsis):
 gtokenized_text.append(tokenizer.encode(syno, add_special_tokens=True))
 gsegments_ids.append([1] * len(gtokenized_text[i]))
 gtokens_tensor.append(torch.tensor([gtokenized_text[i]]))
 gsegments_tensors.append(torch.tensor([gsegments_ids[i]]))


goutputs = []
ghidden_states = []
with torch.no_grad():
 for i in range(len(gtokens_tensor)):
  goutputs.append(model(gtokens_tensor[i], gsegments_tensors[i]))
 # we set `output_hidden_states = True`, the third item will be the
 # hidden states from all layers. See the documentation for more details:
 # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
  ghidden_states.append(goutputs[i][2])




In [69]:
ntokenized_text = []
nsegments_ids = []
ntokens_tensor = []
nsegments_tensors = []

for i, syno in enumerate(nyt_synopsis):
 ntokenized_text.append(tokenizer.encode(syno, add_special_tokens=True))
 nsegments_ids.append([1] * len(ntokenized_text[i]))
 ntokens_tensor.append(torch.tensor([ntokenized_text[i]]))
 nsegments_tensors.append(torch.tensor([nsegments_ids[i]]))


noutputs = []
nhidden_states = []
with torch.no_grad():
 for i in range(len(ntokens_tensor)):
  noutputs.append(model(ntokens_tensor[i], nsegments_tensors[i]))
 # we set `output_hidden_states = True`, the third item will be the
 # hidden states from all layers. See the documentation for more details:
 # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
  nhidden_states.append(noutputs[i][2])


KeyboardInterrupt: ignored

In [66]:
print (len(goodreads_synopsis))
print (len(goodreads))

print (len(nyt_synopsis))
print (len(nyt))

657
82
670
1072
